<a href="https://colab.research.google.com/github/Devimom/LucasDias/blob/main/animes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q google-generativeai

In [11]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Dragon Ball Z,As aventuras de um poderoso guerreiro chamado ...
1,Naruto,Naruto é uma série de mangá escrita e ilustrad...
2,One piece,One Piece é uma série de mangá escrita e ilust...


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Titulo').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('Conteudo').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Conteudo'].value_counts()
    for x_label, grp in df.groupby('Titulo')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Titulo')
_ = plt.ylabel('Conteudo')

In [7]:
model = "models/embedding-001"

In [8]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]


In [13]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Dragon Ball Z,As aventuras de um poderoso guerreiro chamado ...,"[0.042976975, -0.007521984, -0.061024208, 0.01..."
1,Naruto,Naruto é uma série de mangá escrita e ilustrad...,"[-0.00544538, -0.027128238, -0.014095957, 0.03..."
2,One piece,One Piece é uma série de mangá escrita e ilust...,"[-0.015674429, -0.0031357068, -0.052461512, -0..."


In [16]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [18]:
consulta = "Qual a historia de dragon ball Z ?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

As aventuras de um poderoso guerreiro chamado Goku, seu filho Gohan e seus aliados, que se esforçam para defender a Terra das ameaças. Eles são auxiliados por um dragão que concede os desejos de quem quer que reúna as sete Esferas do Dragão.


In [19]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [20]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Imagina só, o Goku, aquele cara superpoderoso, e seu filho Gohan, que também é fera, juntando forças com uma galera irada pra proteger a Terra de uns babacas que querem acabar com tudo. E pra ajudar, eles têm um dragão gente boa que realiza desejos pra quem juntar as sete bolinhas mágicas. É tipo um videogame da vida real, só que muito mais legal!
